In [1]:
import os 
os.chdir('..')
os.chdir('..')
os.getcwd()

'/rhome/f111169109/syner-gpus'

In [2]:
import pandas as pd
import numpy as np
import bs4
import requests
from datasets import load_dataset, Dataset, DatasetDict

# Data Pre-Processing

## Pre-Processing data with pandas

1. Read data
2. Concatenate data 
3. Select dataframe base on columns
4. Drop the row have `NaN` value
5. Drop the row have `twitter` in URL
6. Drop the row is not the url link
7. Save as parquet format for reading and writing data frames efficient

### 1 Read data 

In [3]:
# df1 = pd.read_excel("./datasets/top_5_insurance_brands_june_news_and_twitter_only.xlsx", sheet_name="State_Farm_June_Twitter_and_New")
# df2 = pd.read_excel("./datasets/top_5_insurance_brands_june_news_and_twitter_only.xlsx", sheet_name="Geico_w_Gecko_June_News_and_Twi")
# df3 = pd.read_excel("./datasets/top_5_insurance_brands_june_news_and_twitter_only.xlsx", sheet_name="Progressive_June_Twitter_and_Ne")
# df4 = pd.read_excel("./datasets/top_5_insurance_brands_june_news_and_twitter_only.xlsx", sheet_name="Farmers_Insurance_June_News_and")
# df5 = pd.read_excel("./datasets/top_5_insurance_brands_june_news_and_twitter_only.xlsx", sheet_name="Allstate_June_Twitter_and_News_")

### 2 Concatenate data

In [4]:
# df  = pd.concat([df1, df2, df3, df4, df5], axis=0)

### 3. Select dataframe base on columns

In [5]:
# df = df[["URL", "Headline", "Opening Text", "Hit Sentence", "Source", "Influencer", "Country", "Language"]]

### 4. Drop the row have `NaN` value

In [6]:
# df.dropna(subset=["Opening Text", "Headline", "Hit Sentence", "Source", "Influencer", "Country", "Language"], inplace=True)

### 5. Drop the row have `twitter` in URL

In [7]:
# df = df[~df["URL"].str.contains("twitter")]

### 6. Drop the row is not the url link

In [8]:
# df = df[df["URL"].str.contains("https")]

### 7. Save as parquet format for reading and writing data frames efficient

In [9]:
# df.to_excel("./datasets/processed/top_5_insurance_brands_june_news_and_twitter_only_processed.xlsx", index=False)

## Continue processing data after `url_scraper`

In [10]:
# df1 = pd.read_parquet("./datasets/part0001.parquet")
# df2 = pd.read_parquet("./datasets/part0002.parquet")
# df3 = pd.read_parquet("./datasets/part0003.parquet")
# df4 = pd.read_parquet("./datasets/part0004.parquet")
# df  = pd.concat([df1, df2, df3, df4], axis=0)

In [11]:
# # Easy way to get filter URL 
# df = df[df["HTML source"] != ""]

In [12]:
# df.head(100)

In [13]:
# df.shape

In [14]:
# df.to_parquet("./datasets/processed/dataset.parquet", index=False)

## Reduce data memory

In [15]:
# df = pd.read_parquet("./Datasets/processed/dataset.parquet")

In [16]:
# df.head()

In [17]:
# from tqdm import tqdm

# def get_texts_from_html_source(html_source_text):
#     """
#     Args:
#         html_source_text: str
#     """
#     texts = []
#     for html in tqdm(html_source_text):
#         html = bs4.BeautifulSoup(html, "html.parser")
#         text = html.get_text()
#         texts.append(text)
#     return texts

In [18]:
# texts = get_texts_from_html_source(df["HTML source"])
# df["HTML source"] = texts

In [19]:
# df.head()

In [20]:
# df.dropna(subset=["HTML source"], inplace=True)

In [21]:
# df["HTML source"][1]

In [22]:
# df.to_parquet("./Datasets/processed/dataset_v2.parquet", index=False)

## Format Instruction dataset

In [23]:
dataset = load_dataset("chwenjun225/top_5_insurance_brands_june_news_and_twitter_only")["train"]

In [24]:
def format_instruction_fn(dataframe):
	"""
	Prepare data following the instruct format dataset
	"""
	urls = dataframe["URL"]
	html_sources = dataframe["HTML source"]
	headlines = dataframe["Headline"]
	opening_texts = dataframe["Opening Text"]
	hit_sentences = dataframe["Hit Sentence"]
	sources = dataframe["Source"]
	influencers = dataframe["Influencer"]
	countries = dataframe["Country"]
	languages = dataframe["Language"]
	texts = []
	for url, html_source, headline, opening_text, hit_sentence, source, influencer, country, language in zip(urls, html_sources, headlines, opening_texts, hit_sentences, sources, influencers, countries, languages):
		my_input = html_source
		my_output = {
			"URL": url,
			"Headline": headline, 
			"Opening Text": opening_text, 
			"Hit Sentence": hit_sentence, 
			"Source": source,
			"Influencer": influencer, 
			"Country": country,
			"Language": language 
		}
		text = {
			"instruction": my_input, 
			"output": my_output
		}
		texts.append(text)
	return {"texts": texts}

In [25]:
dataset = dataset.map(format_instruction_fn, batched=True)

Map:   0%|          | 0/5434 [00:00<?, ? examples/s]

In [26]:
dataset["texts"][0]

{'instruction': " Former WKU Lady Toppers star Crystal Kelly becomes Assistant Coach at Kentucky  Skip to contentNewsLivestreamsWeather24/7 WeatherSportsLatest VideosBack to SchoolSt. JudeSpanishAppsHomeElection ResultsInteractive ResultsNewsTrafficStateRegionalNationalInvestigationInternationalAM and MiddayHealthEducationCrime StoppersCrimeLivestreamsWeather24/7 WeatherRadarWeather CamsWeather App TutorialWeather SafetyWhat is a First Alert Weather DayWeather HeadlinesClosings & DelaysSportsSports ConnectionLong GameCourtsideFootball Friday NightsSports VideoNational SportsTalkin' TopsLatest VideosBack to SchoolSt. JudeTelemundo Bowling GreenWBKO NewsletterContact UsMeet the News TeamFollow UsSubmit Photo or VideoJobs with Gray TelevisionJobs with WBKOProgramming ScheduleJob WatchContestsSponsored PagesGood NewsHometown HeroJA People of ActionSOKY Happenings CalendarAppsZeam - News StreamsGray DC BureauInvestigateTVWatching Your WalletDigital MarketingPowerNationCircle CountryFormer W

In [27]:
# Assuming dataset["texts"] is a list of tuples or lists [(input1, output1), (input2, output2), ...]
df = pd.DataFrame(dataset["texts"], columns=["instruction", "output"])

# Display the DataFrame
df.head(5)

,instruction,output
0,Former WKU Lady Toppers star Crystal Kelly be...,"{'Country': 'United States', 'Headline': 'Form..."
1,...,"{'Country': 'United States', 'Headline': 'Myrt..."
2,Fire hardening hasn’t helped insurance ...,"{'Country': 'United States', 'Headline': 'Fire..."
3,How Much Is Homeowners Insurance? Average ...,"{'Country': 'United States', 'Headline': 'The ..."
4,6 Best Life Insurance Companies for Senior...,"{'Country': 'United States', 'Headline': '7 Be..."


In [28]:
df.to_parquet("./Datasets/Dataset_01/Processed/Instruction_dataset.parquet", index=False)

In [29]:
# Upload dataset to HugingFace
dataset = Dataset.from_parquet("./Datasets/Dataset_01/Processed/Instruction_dataset.parquet")
dataset_dict = DatasetDict({"train": dataset})
dataset_dict.push_to_hub("chwenjun225/Instruction_top_5_insurance_brands_june_news_and_twitter_only")

Generating train split: 0 examples [00:00, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/2.67k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/chwenjun225/Instruction_top_5_insurance_brands_june_news_and_twitter_only/commit/fd41fc7822d627c84a2399a665847717c30fff5c', commit_message='Upload dataset', commit_description='', oid='fd41fc7822d627c84a2399a665847717c30fff5c', pr_url=None, pr_revision=None, pr_num=None)

In [30]:
dataset = load_dataset("chwenjun225/Instruction_top_5_insurance_brands_june_news_and_twitter_only")["train"]

Generating train split:   0%|          | 0/5434 [00:00<?, ? examples/s]

In [31]:
# dataset[0].keys()

dict_keys(['instruction', 'output'])

In [34]:
# ds = load_dataset("yahma/alpaca-cleaned")['train']

In [36]:
# PROMPT_DICT = {
#     "prompt_input": (
#         "Below is an instruction that describes a task, paired with an input that provides further context. "
#         "Write a response that appropriately completes the request.\n\n"
#         "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
#     ), 
#     "prompt_no_input": (
#         "Below is an instruction that describes a task. "
#         "Write a response that appropriately completes the request.\n\n"
#         "### Instruction: \n{instruction}\n\n### Response:"
#     ), 
# }

In [37]:
# ann = ds[0]
# ann

{'output': '1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.',
 'input': '',
 'instruction': 'Give three tips for staying healthy.'}

In [39]:
# prompt = PROMPT_DICT["prompt_no_input"].format_map(ann)
# prompt

'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction: \nGive three tips for staying healthy.\n\n### Response:'